# 新版Keras快速教程

---

翻译自[Keras官网教程](https://keras.io/getting_started/intro_to_keras_for_researchers/)。

译者：[金海峰](https://www.zhihu.com/people/jin-hai-feng-68)，Keras团队的一位中国小哥。

## 简介

本教程将帮你掌握Keras和TensorFlow的基本用法和核心概念，其中包括：

- 张量（tensor），变量（variable），梯度（gradient）。
- 通过继承`Layer`类来建立自定义的神经网络层。
- 手写循环来更好地自定义对神经网络训练过程。
- 使用`add_loss()`方法来自定义神经网络层的损失函数。
- 在手写循环训练过程中对评估标准（metrics）进行追踪。
- 使用`tf.function`来编译并加速神经网络的执行过程。
- 神经网络层的运行：训练模式 vs 推断（inference）模式。
- Keras的函数式API。

我们还会使用两个完整的例子来展示如何在实践中使用Keras。

这两个例子分别是：
变分自编码器（Variational Autoencoder）和Hypernetwork。

首先，我们要import tensorflow和keras。

In [1]:
import tensorflow as tf
from tensorflow import keras

## 张量（Tensors）

TensorFlow是一个可微分编程的基础框架。
它的核心功能就是对多维数组（张量）进行操作。
这一点和NumPy类似。

但是，它和NumPy还有很多重要的不同点，例举如下：

- TensorFlow能使用硬件加速器（GPU和TPU）来加速运算。
- TensorFlow能够对各式各样的张量运算表达式进行自动求导。
- TensorFlow能够在多台机器或者一台机器的多个运算设备上进行分布式运算。

我们首先来看一下TensorFlow的一个核心类，`Tensor`。

如下代码建立了一个常量`Tensor`：

In [2]:
x = tf.constant([[5, 2], [1, 3]])
print(x)

tf.Tensor(
[[5 2]
 [1 3]], shape=(2, 2), dtype=int32)


你可以通过调用它的`.numpy()`方法来获取它的值。
该方法会返回一个NumPy数组。

In [3]:
x.numpy()

array([[5, 2],
       [1, 3]], dtype=int32)

与NumPy数组类似，一个`Tensor`对象也有`dtype`和`shape`这两个属性。

In [4]:
print("dtype:", x.dtype)
print("shape:", x.shape)

dtype: <dtype: 'int32'>
shape: (2, 2)


一种常用的简历常量`Tensor`的方法就是使用`tf.ones`和`tf.zeros`，和`np.ones`以及`np.zeros`的用法相同。

In [5]:
print(tf.ones(shape=(2, 1)))
print(tf.zeros(shape=(2, 1)))

tf.Tensor(
[[1.]
 [1.]], shape=(2, 1), dtype=float32)
tf.Tensor(
[[0.]
 [0.]], shape=(2, 1), dtype=float32)


你也可以生成随机的常量`Tensor`。

In [6]:
x = tf.random.normal(shape=(2, 2), mean=0.0, stddev=1.0)

x = tf.random.uniform(shape=(2, 2), minval=0, maxval=10, dtype="int32")


## 变量（Variables）

`Variable`类是一种特殊的张量，它可以用来承载一些可以改变其数值的张量，例如：神经网络的权重。

你可以建立一个`Variable`对象，并指定初始值。

In [7]:
initial_value = tf.random.normal(shape=(2, 2))
a = tf.Variable(initial_value)
print(a)


<tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[ 0.72645265, -0.4716139 ],
       [-1.9029021 , -0.5134245 ]], dtype=float32)>


你可以用`.assign(...)`、`.assign_add(...)`和`.assign_sub(...)`来为标量指定新的值、加上一个值和减去一个值。

In [8]:
new_value = tf.random.normal(shape=(2, 2))
a.assign(new_value)
for i in range(2):
    for j in range(2):
        assert a[i, j] == new_value[i, j]

added_value = tf.random.normal(shape=(2, 2))
a.assign_add(added_value)
for i in range(2):
    for j in range(2):
        assert a[i, j] == new_value[i, j] + added_value[i, j]

## 使用TensorFlow来进行数学运算

用TensorFlow进行数学运算和NumPy非常类似。
最主要的区别是TensorFlow能在GPU和TPU上面运行。

这是一些基本的矩阵运算。

In [9]:
a = tf.random.normal(shape=(2, 2))
b = tf.random.normal(shape=(2, 2))

c = a + b
d = tf.square(c)
e = tf.exp(d)

## 梯度（Gradients）

另一个TensorFlow和NumPy的主要区别就是可以自动对可微的表达式求导并计算梯度。

你可是用`GradientTape`的`tape.watch()`方法来监视一个张量的相关运算，并把要求导的表达式和该张量作为参数传入`tape.gradient(...)`方法，就可以得到梯度了。

In [10]:
a = tf.random.normal(shape=(2, 2))
b = tf.random.normal(shape=(2, 2))

with tf.GradientTape() as tape:
    tape.watch(a)  # 对`a`的所有运算进行记录
    c = tf.sqrt(tf.square(a) + tf.square(b))  # 对`a`进行一些数学运算
    # `c`对`a`的梯度是多少？
    dc_da = tape.gradient(c, a)
    print(dc_da)


tf.Tensor(
[[-0.49744755  0.2795373 ]
 [ 0.90746343  0.94444644]], shape=(2, 2), dtype=float32)


在默认情况下，所有变量都会被自动监视，你不需要特意调用`watch()`。

In [11]:
a = tf.Variable(a)

with tf.GradientTape() as tape:
    c = tf.sqrt(tf.square(a) + tf.square(b))
    dc_da = tape.gradient(c, a)
    print(dc_da)

tf.Tensor(
[[-0.49744755  0.2795373 ]
 [ 0.90746343  0.94444644]], shape=(2, 2), dtype=float32)


如果你把`GradientTape`嵌套起来，就可以求高阶导数啦。

In [12]:
with tf.GradientTape() as outer_tape:
    with tf.GradientTape() as tape:
        c = tf.sqrt(tf.square(a) + tf.square(b))
        dc_da = tape.gradient(c, a)
    d2c_da2 = outer_tape.gradient(dc_da, a)
    print(d2c_da2)


tf.Tensor(
[[0.54047406 1.3902092 ]
 [0.18874693 0.06999791]], shape=(2, 2), dtype=float32)


## Keras中的神经网络层（Layer）

TensorFlow是**可微分编程的基础框架**，主要用于处理张量、变量和梯度。
Keras是**深度学习的用户接口**，主要用于处理神经网络层、模型、优化器、损失函数、评估标准等等。

Keras作为TensorFlow的上层API，使得TensorFlow更加简单用，以提高工程师们的生产力。

`Layer`类是Keras中最基础的一种抽象，表示神经网络的层。
它对权重和一些（在`call()`方法中定义的）计算表达式进行了封装。

下面是一个简单的`Layer`的例子。

In [13]:

class Linear(keras.layers.Layer):
    """y = w.x + b"""

    def __init__(self, units=32, input_dim=32):
        super(Linear, self).__init__()
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(
            initial_value=w_init(shape=(input_dim, units), dtype="float32"),
            trainable=True,
        )
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(
            initial_value=b_init(shape=(units,), dtype="float32"), trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b


你可以把`Layer`的对象当成一个Python函数来用。

In [14]:
# 初始化这个Layer。
linear_layer = Linear(units=4, input_dim=2)

# 我们可以像调用函数一样调用它，传给它一些数据。
y = linear_layer(tf.ones((2, 2)))
assert y.shape == (2, 4)

在`__init__`函数中简历的权重变量会自动被追踪，它们都被放在了`weights`这个属性里面。

In [15]:
assert linear_layer.weights == [linear_layer.w, linear_layer.b]

Keras里面有很多内置的Layer可以使用，比如：`Dense`、`Conv2D`、`LSTM`。

还有一些更神奇的Layer，比如：`Conv3DTranspose`和`ConvLSTM2D`。

这些内置的Layer能帮你在编写神经网络的时候省点事儿。

## 给Layer添加权重（weights）

一种比较方便的方法就是用`self.add_weight()`来添加权重。

In [16]:

class Linear(keras.layers.Layer):
    """y = w.x + b"""

    def __init__(self, units=32):
        super(Linear, self).__init__()
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
        )
        self.b = self.add_weight(
            shape=(self.units,), initializer="random_normal", trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b


# 初始化Layer。它会延迟加载权重。
linear_layer = Linear(4)

# 下面这句才会调用`build(input_shape)`，并添加权重。
y = linear_layer(tf.ones((2, 2)))

## Layer的梯度

你可以通过在`GradientTape`语句里面来调用一个Layer来自动获取它的梯度。
通过梯度，你可以更新一个Layer的权重。
你可以使用优化器来进行自动更新，也可以自己写代码来更新。
当然，如果有必要的话，你也可以在更新权重之前对梯度进行修改。

In [17]:
# 数据准备
(x_train, y_train), _ = tf.keras.datasets.mnist.load_data()
dataset = tf.data.Dataset.from_tensor_slices(
    (x_train.reshape(60000, 784).astype("float32") / 255, y_train)
)
dataset = dataset.shuffle(buffer_size=1024).batch(64)

# 初始化这个线性Layer（在上面的代码中定义的），让它有10个神经元。
linear_layer = Linear(10)

# 初始化一个逻辑损失函数，接受整数值的标签作为输入。
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# 初始化一个优化器。
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)

# 循环数据集中所有的batch
for step, (x, y) in enumerate(dataset):

    # 创建一个GradientTape
    with tf.GradientTape() as tape:

        # 正向传播
        logits = linear_layer(x)

        # 获取对于当前batch的损失函数值
        loss = loss_fn(y, logits)

    # 求损失函数关于权重的梯度
    gradients = tape.gradient(loss, linear_layer.trainable_weights)

    # 更新线性Layer的权重
    optimizer.apply_gradients(zip(gradients, linear_layer.trainable_weights))

    # 输出日志
    if step % 100 == 0:
        print("Step:", step, "Loss:", float(loss))

11493376/11490434 [==============================] - 0s 0us/step
Step: 0 Loss: 2.425366163253784
Step: 100 Loss: 2.2906112670898438
Step: 200 Loss: 2.119081497192383
Step: 300 Loss: 2.087733268737793
Step: 400 Loss: 2.002962112426758
Step: 500 Loss: 1.927724838256836
Step: 600 Loss: 1.8112218379974365
Step: 700 Loss: 1.7492737770080566
Step: 800 Loss: 1.715962529182434
Step: 900 Loss: 1.5601446628570557


## 可训练（trainable）和不可训练（non-trainable）的权重

Layer中的权重可以是可以训练的，也可以是不可训练的。
他们分别存在了`trainalbe_weights`和`non_trainable_weights`属性中。
下面是一个含有不可训练权重的Layer的例子。

In [18]:

class ComputeSum(keras.layers.Layer):
    """Returns the sum of the inputs."""

    def __init__(self, input_dim):
        super(ComputeSum, self).__init__()
        # 建立一个不可训练的权重
        self.total = tf.Variable(initial_value=tf.zeros((input_dim,)), trainable=False)

    def call(self, inputs):
        self.total.assign_add(tf.reduce_sum(inputs, axis=0))
        return self.total


my_sum = ComputeSum(2)
x = tf.ones((2, 2))

y = my_sum(x)
print(y.numpy())  # [2. 2.]

y = my_sum(x)
print(y.numpy())  # [4. 4.]

assert my_sum.weights == [my_sum.total]
assert my_sum.non_trainable_weights == [my_sum.total]
assert my_sum.trainable_weights == []

[2. 2.]
[4. 4.]


## Layer的嵌套使用

Layer可以被递归式地嵌套使用，
这样可以更好地把更多的计算过程包进一个Layer里面。
每个Layer都会自动追踪它所包含的Layer的权重，无论是可训练的还是不可训练的都是。

In [19]:
# 我们建立一个多层感知机（MLP）Layer
# 它包含了我们之前实现的线性Layer


class MLP(keras.layers.Layer):
    """Simple stack of Linear layers."""

    def __init__(self):
        super(MLP, self).__init__()
        self.linear_1 = Linear(32)
        self.linear_2 = Linear(32)
        self.linear_3 = Linear(10)

    def call(self, inputs):
        x = self.linear_1(inputs)
        x = tf.nn.relu(x)
        x = self.linear_2(x)
        x = tf.nn.relu(x)
        return self.linear_3(x)


mlp = MLP()

# 下面这个句首次调用了`mlp`，也就添加了所有权重
y = mlp(tf.ones(shape=(3, 64)))

# 权重是被递归式地追踪的
assert len(mlp.weights) == 6

上面我们手动实现的多层感知机（MLP）与如下使用内置层实现的代码是等价的。

In [20]:
mlp = keras.Sequential(
    [
        keras.layers.Dense(32, activation=tf.nn.relu),
        keras.layers.Dense(32, activation=tf.nn.relu),
        keras.layers.Dense(10),
    ]
)

## 追踪Layer所产生的损失函数

Layer可是在正向传播过程中使用`add_loss()`方法来添加损失函数。
一种很好的把正则化加入损失函数的方法。
子Layer所添加的损失函数也会自动被母Layer所追踪。

下面的例子是把activity正则化加入了损失函数。

In [21]:

class ActivityRegularization(keras.layers.Layer):
    """一个把activity稀疏正则化加入损失函数的Layer。"""

    def __init__(self, rate=1e-2):
        super(ActivityRegularization, self).__init__()
        self.rate = rate

    def call(self, inputs):
        # 使用`add_loss`来添加正则化损失函数，用输入的张量计算
        self.add_loss(self.rate * tf.reduce_sum(inputs))
        return inputs


一个模型只要使用了这个Layer，就会自动把这个正则化的损失函数加入到整体损失函数中。

In [22]:
# 在多层感知机中使用该损失函数。


class SparseMLP(keras.layers.Layer):
    """一系列线性Layer，包含稀疏正则化损失函数。"""

    def __init__(self):
        super(SparseMLP, self).__init__()
        self.linear_1 = Linear(32)
        self.regularization = ActivityRegularization(1e-2)
        self.linear_3 = Linear(10)

    def call(self, inputs):
        x = self.linear_1(inputs)
        x = tf.nn.relu(x)
        x = self.regularization(x)
        return self.linear_3(x)


mlp = SparseMLP()
y = mlp(tf.ones((10, 10)))

print(mlp.losses)  # 一个包含一个float32类型标量的列表

[<tf.Tensor: shape=(), dtype=float32, numpy=0.21000613>]


在每次正向传播开始，最外层的Layer会先把损失函数值清空，以防止把上一次正向传播产生的损失函数加进来。`layer.loss`永远只包含最近一次完成的正向传播所产生的的损失函数。你在自己写训练循环的时候，在用这些损失函数来计算梯度之前，你需要先把他们加起来求和。

In [23]:
# 最近一次正向传播所产生的损失函数
mlp = SparseMLP()
mlp(tf.ones((10, 10)))
assert len(mlp.losses) == 1
mlp(tf.ones((10, 10)))
assert len(mlp.losses) == 1  # 不会包含之前的损失函数

# 我们来演示一下怎样在训练循环中使用这些损失函数

# 准备数据
(x_train, y_train), _ = tf.keras.datasets.mnist.load_data()
dataset = tf.data.Dataset.from_tensor_slices(
    (x_train.reshape(60000, 784).astype("float32") / 255, y_train)
)
dataset = dataset.shuffle(buffer_size=1024).batch(64)

# 一个新的多层感知机对象
mlp = SparseMLP()

# 损失函数和优化器
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)

for step, (x, y) in enumerate(dataset):
    with tf.GradientTape() as tape:

        # 正向传播.
        logits = mlp(x)

        # 对于当前batch的默认的损失函数值
        loss = loss_fn(y, logits)

        # 加上在正向传播过程中所产生的损失函数值
        loss += sum(mlp.losses)

        # 求损失函数关于权重的梯度
        gradients = tape.gradient(loss, mlp.trainable_weights)

    # 对线性Layer的权重进行更新
    optimizer.apply_gradients(zip(gradients, mlp.trainable_weights))

    # 输出日志
    if step % 100 == 0:
        print("Step:", step, "Loss:", float(loss))

Step: 0 Loss: 6.507600784301758
Step: 100 Loss: 2.5422661304473877
Step: 200 Loss: 2.4140090942382812
Step: 300 Loss: 2.3433475494384766
Step: 400 Loss: 2.345116376876831
Step: 500 Loss: 2.342367172241211
Step: 600 Loss: 2.330716848373413
Step: 700 Loss: 2.3208255767822266
Step: 800 Loss: 2.3258860111236572
Step: 900 Loss: 2.3363797664642334


## 追踪训练的评估标准（Metrics）

Keras有很多的内置评估标准，比如`tf.keras.metrics.AUC`和`tf.keras.metrics.PrecisionAtRecall`。
你也可以写一个你自己的评估标准，几行代码就能搞定。

在一个手写的训练循环里面使用评估标准，你要做如下几件事：
- 初始化评估标准，比如`tf.keras.metrics.AUC()`。
- 在每个batch中调用函数`metric.update_state(targets, predictions)`来更新评估标准的统计值。
- 用`metric.results()`来查询评估标准当前的值。
- 在每个epoch结尾或者开始时用`metric.reset_states()`来重置评估标准的值。

下面是一个简单的例子。

In [24]:
# 初始化评估标准对象
accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

# 准备好了我们的Layer、损失函数和优化器
model = keras.Sequential(
    [
        keras.layers.Dense(32, activation="relu"),
        keras.layers.Dense(32, activation="relu"),
        keras.layers.Dense(10),
    ]
)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

for epoch in range(2):
    # 循环数据集里所有的batch
    for step, (x, y) in enumerate(dataset):
        with tf.GradientTape() as tape:
            logits = model(x)
            # 计算当前batch的损失函数值
            loss_value = loss_fn(y, logits)

        # 更新`accuracy`（准确率）评估标准的统计值
        accuracy.update_state(y, logits)

        # 更新模型权重，以最小化损失函数值
        gradients = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(gradients, model.trainable_weights))

        # 输出当前的评估标准值到日志
        if step % 200 == 0:
            print("Epoch:", epoch, "Step:", step)
            print("Total running accuracy so far: %.3f" % accuracy.result())

    # 在epoch结尾重置评估标准的值
    accuracy.reset_states()

Epoch: 0 Step: 0
Total running accuracy so far: 0.125
Epoch: 0 Step: 200
Total running accuracy so far: 0.771
Epoch: 0 Step: 400
Total running accuracy so far: 0.836
Epoch: 0 Step: 600
Total running accuracy so far: 0.862
Epoch: 0 Step: 800
Total running accuracy so far: 0.877
Epoch: 1 Step: 0
Total running accuracy so far: 0.938
Epoch: 1 Step: 200
Total running accuracy so far: 0.938
Epoch: 1 Step: 400
Total running accuracy so far: 0.943
Epoch: 1 Step: 600
Total running accuracy so far: 0.944
Epoch: 1 Step: 800
Total running accuracy so far: 0.944


和`self.add_loss()`类似，你也可以使用`self.add_metric()`来添加任何一个变量进去作为评估标准。可以用`layer.reset_metrics()`来重置某一层或整个模型的评估标准的值。


## 函数编译

默认执行时我们使用的是Eager模式，调试程序比较方便。
与之对应的静态图模式则在执行时更快。
你只要把一个函数用`@tf.function`这个装饰器装饰一下，那个函数就会以静态图模式执行了。

In [25]:
# 准备好模型、损失函数和优化器
model = keras.Sequential(
    [
        keras.layers.Dense(32, activation="relu"),
        keras.layers.Dense(32, activation="relu"),
        keras.layers.Dense(10),
    ]
)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

# 建立一个训练函数，每次执行一个batch


@tf.function  # 用这句来加速
def train_on_batch(x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = loss_fn(y, logits)
        gradients = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(gradients, model.trainable_weights))
    return loss


# 准备数据
(x_train, y_train), _ = tf.keras.datasets.mnist.load_data()
dataset = tf.data.Dataset.from_tensor_slices(
    (x_train.reshape(60000, 784).astype("float32") / 255, y_train)
)
dataset = dataset.shuffle(buffer_size=1024).batch(64)

for step, (x, y) in enumerate(dataset):
    loss = train_on_batch(x, y)
    if step % 100 == 0:
        print("Step:", step, "Loss:", float(loss))

Step: 0 Loss: 2.3842484951019287
Step: 100 Loss: 0.4142776131629944
Step: 200 Loss: 0.2205623984336853
Step: 300 Loss: 0.5111865997314453
Step: 400 Loss: 0.3792179822921753
Step: 500 Loss: 0.22862757742404938
Step: 600 Loss: 0.14353641867637634
Step: 700 Loss: 0.25944337248802185
Step: 800 Loss: 0.17433446645736694
Step: 900 Loss: 0.10836410522460938


## 训练模式和推断（inference）模式

某些神经网络的层在训练和推断的时候采用的是不同的计算公式，也就是说，他们在训练和推断的时候做的事情是不一样的。
对于这种情况，我们有一种标准操作来帮你拿到现在究竟是在训练还是在推断这个信息。
那就是在`call`函数里面有一个布尔型参数叫`training`。

在你自定义层的`call`函数里面使用这个参数，可以帮你正确地实现好该层在训练和推断时候的不同表现。


In [26]:

class Dropout(keras.layers.Layer):
    def __init__(self, rate):
        super(Dropout, self).__init__()
        self.rate = rate

    def call(self, inputs, training=None):
        if training:
            return tf.nn.dropout(inputs, rate=self.rate)
        return inputs


class MLPWithDropout(keras.layers.Layer):
    def __init__(self):
        super(MLPWithDropout, self).__init__()
        self.linear_1 = Linear(32)
        self.dropout = Dropout(0.5)
        self.linear_3 = Linear(10)

    def call(self, inputs, training=None):
        x = self.linear_1(inputs)
        x = tf.nn.relu(x)
        x = self.dropout(x, training=training)
        return self.linear_3(x)


mlp = MLPWithDropout()
y_train = mlp(tf.ones((2, 2)), training=True)
y_test = mlp(tf.ones((2, 2)), training=False)

## 用函数式API来建立模型

要建立模型的话，你不一定非要用面向对象的方法。我们之前所看到的Layer是可以用函数式的方法来组合到一起的，我们称之为函数式API。

In [27]:
# 用`Input`对象来指定输入的尺寸和类型，就像我们给变量定义一个类型一样
# `shape`是用来描述单个样本的尺寸的，不包含batch大小这个维度
# 函数式API旨在定义单个样本在模型里面的变化过程
# 模型会自动把这些对单个样本的操作打包成对batch的操作，然后按batch来处理数据
inputs = tf.keras.Input(shape=(16,), dtype="float32")

# 我们把这些Layer当函数调用，并输入这些标记着变量类型和尺寸的对象
# 我们会收到一个标记着新的尺寸和类型的对象
x = Linear(32)(inputs)  # 使用我们之前定义的线性Layer
x = Dropout(0.5)(x)  # 使用我们之前定义的Dropout层
outputs = Linear(10)(x)

# 一个函数式的`Model`对象可以用它的输入和输出来进行定义
# 一个`Model`对象本身也是一个`Layer`
model = tf.keras.Model(inputs, outputs)

# 一个函数式的模型在我们用数据调用它之前就已经有了权重
# 因为我们建立模型时就定义了它的输入尺寸，它也就可以推理出所有权重的尺寸了
assert len(model.weights) == 4

# 我们来试着把一些数据输入到模型里面
y = model(tf.ones((2, 16)))
assert y.shape == (2, 10)

# 我们也可以给`__call__`函数发一个`training`的参数
# 这个参数会一直深入传递到Dropout层
y = model(tf.ones((2, 16)), training=True)

函数式API比继承的方式更简洁，并且还有一些别的优势，即函数式编程相比于面向对象编程的一些优势。
但是，函数式API只能用来定义一些有向无环图（DAG），对于循环神经网络来讲，我们就必须使用继承的方式来实现。

[这里](https://keras.io/guides/functional_api/)有更详尽的函数式API教程。

在你建立模型的过程中，可能通常要使用函数式和继承这两种建模方法的结合。

`Model`类的另一个比较好的特性是有`fit()`和`evaluate()`这些内置的函数。你可以继承`Model`类，就像我们继承`Layer`类一样，然后重载这些函数来实现你自己的训练和评估循环。

## 完整例子1：变分自编码器（Variational Autoencoder, VAE）

我们先来总结一下我们已经学到的内容：

- 一个`Layer`对象可以封装一些状态值（在`__init__`或者`build`里面定义的权重等等）和一些计算式（在`call`里面定义的）。
- Layer可以递归式地组合在一起来构建更大的包含更多计算的Layer。
- 你可以自定义的训练循环，你只需要开启`GradientTape`，在其内部调用你的模型来获取梯度，并传递给优化器即可。
- 你可以用`@tf.function`装饰器来给你的训练循环加速。
- 可以用`self.add_loss()`来给Layer添加损失函数，一般是用来添加正则化的。

我们接下来就把这些内容一起放进一个完整的例子里。我们来写一变分自编码器（Variational Autoencoder, VAE），并且使用MNIST数据集来进行训练。

VAE会继承Layer类，并嵌套调用其他Layer。还会使用KL散度作为正则化的损失函数。


下面的代码就是用来建立模型的。

首先，我们需要一个`Encoder`编码器类，它把一个MNIST手写数字图片来对应到一个在潜在空间（latent space）里面的三元组`(z_mean, z_log_var, z)`，过程中使用了一个`Sampling`采样层。

In [28]:
from tensorflow.keras import layers


class Sampling(layers.Layer):
    """使用`(z_mean, z_log_var)`来采样获得对一个数字的编码z"""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon


class Encoder(layers.Layer):
    """把一个数字对应到一个三元组`(z_mean, z_log_var, z)`"""

    def __init__(self, latent_dim=32, intermediate_dim=64, **kwargs):
        super(Encoder, self).__init__(**kwargs)
        self.dense_proj = layers.Dense(intermediate_dim, activation=tf.nn.relu)
        self.dense_mean = layers.Dense(latent_dim)
        self.dense_log_var = layers.Dense(latent_dim)
        self.sampling = Sampling()

    def call(self, inputs):
        x = self.dense_proj(inputs)
        z_mean = self.dense_mean(x)
        z_log_var = self.dense_log_var(x)
        z = self.sampling((z_mean, z_log_var))
        return z_mean, z_log_var, z


下一步，我们用一个`Decoder`解码器类来把潜在空间的坐标对应回到一个MNIST数字图片。

In [29]:

class Decoder(layers.Layer):
    """把编码成的向量z转化回原来的数字图片"""

    def __init__(self, original_dim, intermediate_dim=64, **kwargs):
        super(Decoder, self).__init__(**kwargs)
        self.dense_proj = layers.Dense(intermediate_dim, activation=tf.nn.relu)
        self.dense_output = layers.Dense(original_dim, activation=tf.nn.sigmoid)

    def call(self, inputs):
        x = self.dense_proj(inputs)
        return self.dense_output(x)


最后，我们的`VariationalAutoEncoder`变分自编码器类会把编码器和解码器串起来，然后用`add_loss()`来加入KL散度正则化的损失函数。

In [30]:

class VariationalAutoEncoder(layers.Layer):
    """把编码器和解码器串起来形成一个完成的模型用于训练"""

    def __init__(self, original_dim, intermediate_dim=64, latent_dim=32, **kwargs):
        super(VariationalAutoEncoder, self).__init__(**kwargs)
        self.original_dim = original_dim
        self.encoder = Encoder(latent_dim=latent_dim, intermediate_dim=intermediate_dim)
        self.decoder = Decoder(original_dim, intermediate_dim=intermediate_dim)

    def call(self, inputs):
        z_mean, z_log_var, z = self.encoder(inputs)
        reconstructed = self.decoder(z)
        # 把KL散度正则化加入损失函数
        kl_loss = -0.5 * tf.reduce_mean(
            z_log_var - tf.square(z_mean) - tf.exp(z_log_var) + 1
        )
        self.add_loss(kl_loss)
        return reconstructed


现在我们来写一个训练循环，我们的训练过程会使用`@tf.function`来编译成静态图来加速。

In [31]:
# 初始化模型
vae = VariationalAutoEncoder(original_dim=784, intermediate_dim=64, latent_dim=32)

# 损失函数和优化器
loss_fn = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

# 准备数据
(x_train, _), _ = tf.keras.datasets.mnist.load_data()
dataset = tf.data.Dataset.from_tensor_slices(
    x_train.reshape(60000, 784).astype("float32") / 255
)
dataset = dataset.shuffle(buffer_size=1024).batch(32)


@tf.function
def training_step(x):
    with tf.GradientTape() as tape:
        reconstructed = vae(x)  # 计算出重建的输入图片
        # 计算损失函数值
        loss = loss_fn(x, reconstructed)
        loss += sum(vae.losses)  # 加上KL散度的损失函数
    # 更新VAE的权重
    grads = tape.gradient(loss, vae.trainable_weights)
    optimizer.apply_gradients(zip(grads, vae.trainable_weights))
    return loss


losses = []  # 用于记录过程中产生的损失函数值
for step, x in enumerate(dataset):
    loss = training_step(x)
    # 输出日志
    losses.append(float(loss))
    if step % 100 == 0:
        print("Step:", step, "Loss:", sum(losses) / len(losses))

    # 1000步之后停止
    # 把模型训练至收敛就当成留给你的练习题了
    if step >= 1000:
        break

Step: 0 Loss: 0.31538477540016174
Step: 100 Loss: 0.12489071001510808
Step: 200 Loss: 0.09933947263962001
Step: 300 Loss: 0.0893392476726608
Step: 400 Loss: 0.08442602329829388
Step: 500 Loss: 0.08138602710293677
Step: 600 Loss: 0.07897867765680526
Step: 700 Loss: 0.0776745897960901
Step: 800 Loss: 0.07644307679879234
Step: 900 Loss: 0.07552914691470836
Step: 1000 Loss: 0.07460356648866233


由此可见，用Keras来建立和训练这样的模型是很快很容易实现的。

现在你可能觉得上面的代码还是有点啰嗦，这是因为我们每个细节都是亲自用代码实现的。这让我们有了最大的自由度，但同时也增加了我们的工作量。

我们看看用函数式API来实现的话怎么样。

In [32]:
original_dim = 784
intermediate_dim = 64
latent_dim = 32

# 编码器
original_inputs = tf.keras.Input(shape=(original_dim,), name="encoder_input")
x = layers.Dense(intermediate_dim, activation="relu")(original_inputs)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()((z_mean, z_log_var))
encoder = tf.keras.Model(inputs=original_inputs, outputs=z, name="encoder")

# 解码器
latent_inputs = tf.keras.Input(shape=(latent_dim,), name="z_sampling")
x = layers.Dense(intermediate_dim, activation="relu")(latent_inputs)
outputs = layers.Dense(original_dim, activation="sigmoid")(x)
decoder = tf.keras.Model(inputs=latent_inputs, outputs=outputs, name="decoder")

# VAE模型
outputs = decoder(z)
vae = tf.keras.Model(inputs=original_inputs, outputs=outputs, name="vae")

# 添加KL散度正则化损失函数
kl_loss = -0.5 * tf.reduce_mean(z_log_var - tf.square(z_mean) - tf.exp(z_log_var) + 1)
vae.add_loss(kl_loss)

Much more concise, right?

By the way, Keras also features built-in training & evaluation loops on its `Model` class
(`fit()` and `evaluate()`). Check it out:

这样一来就简洁多了。

另外，Keras还有内置的训练和评估循环，是`Model`类的方法（`fit()`和`evaluate()`）。代码如下：

In [33]:
# 损失函数和优化器
loss_fn = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

# 准备数据
(x_train, _), _ = tf.keras.datasets.mnist.load_data()
dataset = tf.data.Dataset.from_tensor_slices(
    x_train.reshape(60000, 784).astype("float32") / 255
)
dataset = dataset.map(lambda x: (x, x))  # 用x_train同时作为输入和输出目标
dataset = dataset.shuffle(buffer_size=1024).batch(32)

# 配置模型，准备训练
vae.compile(optimizer, loss=loss_fn)

# 对模型进行训练
vae.fit(dataset, epochs=1)

1875/1875 [==============================] - 6s 3ms/step - loss: 0.0714


使用函数式API的`fit`方法把我们的代码从之前的65行缩短到了25行（包括了模型的定义和训练）。
Keras的设计原则是最大化你的生产力，同时也可以让你像我们两段之前的那个例子一样，自己实现和控制每个细节。

## 完整例子2：超网络（Hypernetworks）

我们来看一下另一个实验：超网络（Hypernetworks）。

超网络是一种特殊的深度神经网络，它的权重是用另一个神经网络生成出来的。（通常另一个神经网络要更小一些）。

我们来实现一个非常简单的超网络。我们用一个两层的神经网络来输出另一个三层的神经网络的权重。


In [34]:
import numpy as np

input_dim = 784
classes = 10

# 这个是我们用来预测数据标签的模型
outer_model = keras.Sequential(
    [keras.layers.Dense(64, activation=tf.nn.relu), keras.layers.Dense(classes),]
)

# 它不需要建立自己的权重（而是另一个网络输出的），所以我们把它设置成已经完成了build过程
# 这样我们调用上面的outer_model的时候，就不会自动产生新的权重了
for layer in outer_model.layers:
    layer.built = True

# 算是下有多少个权重值需要另一个网络输出
# 每个outer_model的层都需要output_dim * input_dim + output_dim个权重值。
num_weights_to_generate = (classes * 64 + classes) + (64 * input_dim + 64)

# 下面这个模型是用来输出outer_model的权重的。
inner_model = keras.Sequential(
    [
        keras.layers.Dense(16, activation=tf.nn.relu),
        keras.layers.Dense(num_weights_to_generate, activation=tf.nn.sigmoid),
    ]
)

在训练循环里，对于每个batch，我们要做如下几件事：

- 用`inner_model`来生成权重数组`weights_pred`。
- 修改权重数组的尺寸，变成和`outer_model`所需要的权重尺寸一样。
- `outer_model`运行一次正向传播来输出对MNIST图片的预测结果。
- 运行反向传播来更新`inner_model`自身的权重，来让分类损失函数最小化。

In [35]:
# 损失函数和优化器
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)

# 准备数据
(x_train, y_train), _ = tf.keras.datasets.mnist.load_data()
dataset = tf.data.Dataset.from_tensor_slices(
    (x_train.reshape(60000, 784).astype("float32") / 255, y_train)
)

# 我们把batch大小设置为1
dataset = dataset.shuffle(buffer_size=1024).batch(1)


@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        # 输出outer_model的权重
        weights_pred = inner_model(x)

        # 改变输出权重的尺寸，编程和w、b一样的尺寸给outer_model.
        # 第0层的kernel矩阵
        start_index = 0
        w0_shape = (input_dim, 64)
        w0_coeffs = weights_pred[:, start_index : start_index + np.prod(w0_shape)]
        w0 = tf.reshape(w0_coeffs, w0_shape)
        start_index += np.prod(w0_shape)
        # 第0层的偏移向量
        b0_shape = (64,)
        b0_coeffs = weights_pred[:, start_index : start_index + np.prod(b0_shape)]
        b0 = tf.reshape(b0_coeffs, b0_shape)
        start_index += np.prod(b0_shape)
        # 第1层的kernel矩阵
        w1_shape = (64, classes)
        w1_coeffs = weights_pred[:, start_index : start_index + np.prod(w1_shape)]
        w1 = tf.reshape(w1_coeffs, w1_shape)
        start_index += np.prod(w1_shape)
        # 第1层的偏移向量
        b1_shape = (classes,)
        b1_coeffs = weights_pred[:, start_index : start_index + np.prod(b1_shape)]
        b1 = tf.reshape(b1_coeffs, b1_shape)
        start_index += np.prod(b1_shape)

        # 把这些权重赋给outer_model
        outer_model.layers[0].kernel = w0
        outer_model.layers[0].bias = b0
        outer_model.layers[1].kernel = w1
        outer_model.layers[1].bias = b1

        # 用outer_model进行推断
        preds = outer_model(x)
        loss = loss_fn(y, preds)

    # 只更新inner_model的权重
    grads = tape.gradient(loss, inner_model.trainable_weights)
    optimizer.apply_gradients(zip(grads, inner_model.trainable_weights))
    return loss


losses = []  # 用于记录过程中产生的损失函数值
for step, (x, y) in enumerate(dataset):
    loss = train_step(x, y)

    # 输出日志
    losses.append(float(loss))
    if step % 100 == 0:
        print("Step:", step, "Loss:", sum(losses) / len(losses))

    # 1000步之后停止
    # 把模型训练至收敛就当成留给你的练习题了
    if step >= 1000:
        break

Step: 0 Loss: 2.366020917892456
Step: 100 Loss: 2.3394237100561655
Step: 200 Loss: 2.1981412654937205
Step: 300 Loss: 2.0647767751208472
Step: 400 Loss: 1.937349613778005
Step: 500 Loss: 1.884672182062808
Step: 600 Loss: 1.822935720475254
Step: 700 Loss: 1.749699467633804
Step: 800 Loss: 1.703513056691031
Step: 900 Loss: 1.634726283406388
Step: 1000 Loss: 1.5674246165387933


Keras是一个强有力的生产力工具，让你能轻松实现任何的科研想法。试想一下，你可能可以在一天之内尝试25个不同的想法（每20分钟一个）！

Keras的宗旨之一就是从以最快的速度把想法变成结果，我们相信这是做出伟大科研成果的关键。

希望你喜欢这篇教程。记得告诉我们你用Keras做过什么有趣的事情。